In [64]:
import pandas as pd
import numpy as np
import glob
import re

In [65]:
OUTPUT_DIR = "04_transform_aec_electorates"

In [66]:
exported_files = glob.glob("03_extract_aec_electorates/*.csv")

In [67]:
df = pd.read_csv(exported_files[0])

# Fix Column Headers

In [68]:
def fix_column_headers(df):
    df.columns = [re.sub(r"[^\w]", "_", x.lower()) for x in df.columns]
    return df

In [69]:
df = fix_column_headers(df)
df.head()

,state,locality_suburb,postcode,electorate,redistributed_electorate,other_locality_s_
0,SA,ADELAIDE,5000,Adelaide,NaN,NaN
1,SA,ALLENBY GARDENS,5009,Adelaide,NaN,NaN
2,SA,ANGLE PARK,5010,Adelaide,NaN,NaN
3,SA,ASHFORD,5035,Adelaide,NaN,NaN
4,SA,BLAIR ATHOL,5084,Adelaide,NaN,NaN


In [70]:
df.describe(include="all")

,state,locality_suburb,postcode,electorate,redistributed_electorate,other_locality_s_
count,81,81,81.000000,81,4.0,4.0
unique,3,75,NaN,3,NaN,NaN
top,SA,1234,NaN,Adelaide,NaN,NaN
freq,73,4,NaN,73,NaN,NaN
mean,NaN,NaN,4607.790123,NaN,1234.0,1234.0
std,NaN,NaN,1343.346127,NaN,0.0,0.0
min,NaN,NaN,3.000000,NaN,1234.0,1234.0
25%,NaN,NaN,5008.000000,NaN,1234.0,1234.0
50%,NaN,NaN,5034.000000,NaN,1234.0,1234.0
75%,NaN,NaN,5081.000000,NaN,1234.0,1234.0


In [71]:
df.loc[df["state"] != "SA"]

,state,locality_suburb,postcode,electorate,redistributed_electorate,other_locality_s_
20,1234,1234,1234,1234,1234.0,1234.0
21,1,2,3,4,NaN,NaN
42,1234,1234,1234,1234,1234.0,1234.0
43,1,2,3,4,NaN,NaN
64,1234,1234,1234,1234,1234.0,1234.0
65,1,2,3,4,NaN,NaN
79,1234,1234,1234,1234,1234.0,1234.0
80,1,2,3,4,NaN,NaN


In [74]:
def drop_numeric_rows_in_column(df, column):
    return df.loc[~df[column].apply(lambda x: re.sub(r"[^a-z\s]", "", x.lower()).strip()).replace("", np.nan).isnull()]

In [75]:
df = drop_numeric_rows_in_column(df, "state")

In [76]:
df["state"].value_counts()

SA    73
Name: state, dtype: int64

In [77]:
df.describe(include="all")

,state,locality_suburb,postcode,electorate,redistributed_electorate,other_locality_s_
count,73,73,73.000000,73,0.0,0.0
unique,1,73,NaN,1,NaN,NaN
top,SA,HILTON,NaN,Adelaide,NaN,NaN
freq,73,1,NaN,73,NaN,NaN
mean,NaN,NaN,5044.972603,NaN,NaN,NaN
std,NaN,NaN,30.667560,NaN,NaN,NaN
min,NaN,NaN,5000.000000,NaN,NaN,NaN
25%,NaN,NaN,5010.000000,NaN,NaN,NaN
50%,NaN,NaN,5035.000000,NaN,NaN,NaN
75%,NaN,NaN,5081.000000,NaN,NaN,NaN


In [78]:
def drop_redundent_cols(df):
    return df.drop(["redistributed_electorate", "other_locality_s_"], axis = 1)

In [79]:
df = drop_redundent_cols(df)

In [80]:
df.head()

,state,locality_suburb,postcode,electorate
0,SA,ADELAIDE,5000,Adelaide
1,SA,ALLENBY GARDENS,5009,Adelaide
2,SA,ANGLE PARK,5010,Adelaide
3,SA,ASHFORD,5035,Adelaide
4,SA,BLAIR ATHOL,5084,Adelaide


In [81]:
df.dtypes

state              object
locality_suburb    object
postcode            int64
electorate         object
dtype: object

In [82]:
def title_case_locality_suburb(df):
    df["locality_suburb"] = df["locality_suburb"].astype(str).str.title()
    return df

In [83]:
df = title_case_locality_suburb(df)
df.head()

,state,locality_suburb,postcode,electorate
0,SA,Adelaide,5000,Adelaide
1,SA,Allenby Gardens,5009,Adelaide
2,SA,Angle Park,5010,Adelaide
3,SA,Ashford,5035,Adelaide
4,SA,Blair Athol,5084,Adelaide


In [84]:
def transform_electorate_df(file_path):
    df = pd.read_csv(file_path)
    df = fix_column_headers(df)
    df = drop_numeric_rows_in_column(df, "state")
    df = drop_numeric_rows_in_column(df, "locality_suburb")
    df = drop_numeric_rows_in_column(df, "electorate")
    df = drop_redundent_cols(df)
    df = title_case_locality_suburb(df)
    
    df = df[["electorate", "locality_suburb", "postcode", "state"]]
    
    return df

In [85]:
transformed_dfs = list()

for exported_file in exported_files:
    transformed_dfs.append(transform_electorate_df(exported_file))



In [86]:
electorates_df = pd.concat(transformed_dfs)

In [87]:
electorates_df.shape

(17715, 4)

In [88]:
electorates_df.describe(include="all")

,electorate,locality_suburb,postcode,state
count,17715,17715,17715,17715
unique,151,15631,2753,8
top,Lingiari,Red Hill,0872,NSW
freq,1526,10,474,5024


check out the postcodes

In [89]:
try:
    pd.to_numeric(electorates_df["postcode"], errors = "raise")
except ValueError:
    raise("Postcodes not expected values")

In [90]:
electorates_df["postcode"] = electorates_df["postcode"].astype(str).str.split(".").str[0]

In [92]:
electorates_df.head()

,electorate,locality_suburb,postcode,state
0,Adelaide,Adelaide,5000,SA
1,Adelaide,Allenby Gardens,5009,SA
2,Adelaide,Angle Park,5010,SA
3,Adelaide,Ashford,5035,SA
4,Adelaide,Blair Athol,5084,SA


In [93]:
def convert_column_to_fk(df, column):
    global OUTPUT_DIR
    
    values = sorted(list(df[column].unique()))
    ids = range(1, len(values) + 1)
    
    fk_df = pd.DataFrame(values, index = ids).reset_index()
    fk_df.columns = ["id", column]
    fk_df.to_csv(f"{OUTPUT_DIR}/{column}.csv", index = False)
    
    df[column] = df[column].replace(dict(zip(values, ids)))
    df = df.rename(columns = {column: f"{column}_id"})

    return df

In [94]:
for col in ["electorate", "locality_suburb", "postcode", "state"]:
    electorates_df = convert_column_to_fk(electorates_df, col)

In [95]:
electorates_df.head()

,electorate_id,locality_suburb_id,postcode_id,state_id
0,1,51,1813,5
1,1,165,1817,5
2,1,279,1818,5
3,1,453,1838,5
4,1,1364,1873,5


In [96]:
electorates_df.shape

(17715, 4)

In [97]:
electorates_df.drop_duplicates().shape

(17668, 4)

In [98]:
electorates_df.to_csv(f"{OUTPUT_DIR}/mapping.csv",index = False)